<a href="https://colab.research.google.com/github/Akshata1992/Implement_ResearchPaper/blob/main/VGG16_TensorFlow_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementing VGG16 network research paper in tensorflow framework for cifar10 dataset

In [1]:
#import all necessary libraries
import tensorflow as tf
print(tf.__version__)

2.3.0


In [14]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

In [3]:
#Load the cifar dataset and normalize the dataset

data = tf.keras.datasets.cifar10
(x_train,y_train),(x_test,y_test) = data.load_data()
x_train,x_test = x_train/255.0,x_test/255.0
y_train,y_test = y_train.flatten(),y_test.flatten()
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

170500096/170498071 [==============================] - 3s 0us/step
(50000, 32, 32, 3)
(50000,)
(10000, 32, 32, 3)
(10000,)


In [4]:
input_shape = x_train[0].shape
C = len(set(y_train))
print('Number of classes: ',C)
print('Input shape: ',input_shape)

Number of classes:  10
Input shape:  (32, 32, 3)


In [13]:
#Data Augmentation
data_gen = ImageDataGenerator()
tr_data = ImageDataGenerator(rotation_range=2,horizontal_flip=True,zoom_range=0.1)
ts_data = ImageDataGenerator(rotation_range=2,horizontal_flip=True,zoom_range=0.1)

#fit the transform
tr_data.fit(x_train)
ts_data.fit(x_test)

In [7]:
#Build the model from scratch using Keras Functional API

#input layer
i = Input(shape=input_shape)

#ConvNets
x = Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(i)
x = Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)
x = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x)

x = Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)
x = Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)
x = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x)

x = Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)
x = Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)
x = Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)
x = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x)

x = Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)
x = Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)
x = Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)
x = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x)

x = Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)
x = Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)
x = Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)
x = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x)

#Flatten the layers
x = Flatten()(x)

#Fully Connected layers
x = Dense(4096,activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(4096,activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(C,activation='softmax')(x)

model = Model(i,x)
model.summary()

Model: "functional_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 8, 8, 128)        

In [19]:
#Compile the model
opt = Adam(lr=0.001)
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [20]:
#Checkpoint and earlystopping
checkpoint = ModelCheckpoint('VGG16_weights_1.h5',monitor='val_acc',verbose=1,save_best_only=True,
                             save_weights_only=False,mode='auto')

early = EarlyStopping(monitor='val_acc',min_delta=0,patience=20,mode='auto',verbose=1)

In [25]:
#fit the model
"""
hist = model.fit(tr_data.flow(x_train,y_train,batch_size=100),epochs=100,callbacks=[checkpoint,early],
                 validation_data=ts_data.flow(x_test,y_test),steps_per_epoch=100,validation_steps=10)
"""

hist = model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100)

Epoch 1/100
1563/1563 [==============================] - 50s 32ms/step - loss: 2.3028 - accuracy: 0.0982 - val_loss: 2.3023 - val_accuracy: 0.1094
Epoch 2/100
1563/1563 [==============================] - 50s 32ms/step - loss: 2.3028 - accuracy: 0.0969 - val_loss: 2.3037 - val_accuracy: 0.0906
Epoch 3/100
1563/1563 [==============================] - 49s 32ms/step - loss: 2.3028 - accuracy: 0.0967 - val_loss: 2.3023 - val_accuracy: 0.1219
Epoch 4/100
1563/1563 [==============================] - 49s 32ms/step - loss: 2.3027 - accuracy: 0.0991 - val_loss: 2.3029 - val_accuracy: 0.0844
Epoch 5/100
 843/1563 [===============>..............] - ETA: 22s - loss: 2.3028 - accuracy: 0.0972

KeyboardInterrupt: ignored

In [26]:
#Visualization
plt.plot(hist.history['loss'],label='Training Loss')
plt.plot(hist.history['val_loss'],label='Validation Loss')
plt.legend()

NameError: ignored

In [27]:
plt.plot(hist.history['accuracy'],label='Training Accuracy')
plt.plot(hist.history['val_accuracy'],label='Validation Acuuracy')

NameError: ignored

In [ ]:
# Plot confusion matrix
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
  """
  This function prints and plots the confusion matrix.
  Normalization can be applied by setting `normalize=True`.
  """
  if normalize:
      cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
      print("Normalized confusion matrix")
  else:
      print('Confusion matrix, without normalization')

  print(cm)

  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  fmt = '.2f' if normalize else 'd'
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, format(cm[i, j], fmt),
               horizontalalignment="center",
               color="white" if cm[i, j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()


p_test = model.predict(x_test).argmax(axis=1)
cm = confusion_matrix(y_test, p_test)
plot_confusion_matrix(cm, list(range(10)))